<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Copia_de_Bayesian_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
dores bayesian learning


In [ ]:
# Importar las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive_output, HBox, VBox, Layout

# Estilo para los gráficos
plt.style.use('seaborn-v0_8-whitegrid')

# --- Simulador 1: Filtro de Kalman (Versión sin Matrices) ---

def simulate_kalman(n_trials, n_preexposure, initial_uncertainty, volatility, noise):
    """
    Simula el aprendizaje de un solo estímulo usando el Filtro de Kalman.
    """
    total_trials = n_preexposure + n_trials

    # Inicialización de variables
    w_hat = np.zeros(total_trials + 1)  # Estimación del peso (media)
    sigma_sq = np.zeros(total_trials + 1) # Incertidumbre (varianza)
    kalman_gain = np.zeros(total_trials) # Tasa de aprendizaje

    sigma_sq[0] = initial_uncertainty

    # Fase de Preexposición (Inhibición Latente)
    for t in range(n_preexposure):
        reward = 0.0 # No hay recompensa

        # 1. Calcular Error de Predicción
        delta = reward - w_hat[t]

        # Incertidumbre predicha para este ensayo
        predicted_sigma_sq = sigma_sq[t] + volatility

        # 2. Calcular Ganancia de Kalman
        k = predicted_sigma_sq / (predicted_sigma_sq + noise)
        kalman_gain[t] = k

        # 3. Actualizar Estimación
        w_hat[t+1] = w_hat[t] + k * delta

        # 4. Actualizar Incertidumbre
        sigma_sq[t+1] = (1 - k) * predicted_sigma_sq

    # Fase de Condicionamiento
    for t in range(n_preexposure, total_trials):
        reward = 1.0 # Hay recompensa

        # 1. Calcular Error de Predicción
        delta = reward - w_hat[t]

        # Incertidumbre predicha para este ensayo
        predicted_sigma_sq = sigma_sq[t] + volatility

        # 2. Calcular Ganancia de Kalman
        k = predicted_sigma_sq / (predicted_sigma_sq + noise)
        kalman_gain[t] = k

        # 3. Actualizar Estimación
        w_hat[t+1] = w_hat[t] + k * delta

        # 4. Actualizar Incertidumbre
        sigma_sq[t+1] = (1 - k) * predicted_sigma_sq

    # Crear los gráficos
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

    # Gráfico 1: Peso Asociativo
    ax1.plot(w_hat, 'b-', label='Peso Estimado (ŵ)')
    ax1.axhline(1, color='r', linestyle='--', label='Valor Real de la Recompensa')
    if n_preexposure > 0:
        ax1.axvline(n_preexposure, color='k', linestyle=':', label='Inicio del Condicionamiento')
    ax1.set_title('Evolución de la Fuerza Asociativa (ŵ)', fontsize=14)
    ax1.set_ylabel('Fuerza Asociativa')
    ax1.legend()
    ax1.set_ylim(-0.2, 1.2)

    # Gráfico 2: Incertidumbre
    ax2.plot(sigma_sq, 'g-', label='Incertidumbre (σ²)')
    if n_preexposure > 0:
        ax2.axvline(n_preexposure, color='k', linestyle=':', label='Inicio del Condicionamiento')
    ax2.set_title('Evolución de la Incertidumbre (σ²)', fontsize=14)
    ax2.set_ylabel('Incertidumbre (Varianza)')
    ax2.legend()

    # Gráfico 3: Ganancia de Kalman
    ax3.plot(kalman_gain, 'm-', label='Ganancia de Kalman (k)')
    if n_preexposure > 0:
        ax3.axvline(n_preexposure, color='k', linestyle=':', label='Inicio del Condicionamiento')
    ax3.set_title('Evolución de la Tasa de Aprendizaje (Ganancia de Kalman)', fontsize=14)
    ax3.set_ylabel('Tasa de Aprendizaje (k)')
    ax3.set_xlabel('Ensayos')
    ax3.legend()
    ax3.set_ylim(0, 1)

    plt.tight_layout()
    plt.show()

# Creación de los Widgets para Kalman
style = {'description_width': 'initial'}
kalman_widgets = VBox([
    widgets.IntSlider(value=50, min=10, max=200, step=1, description='Ensayos de Condicionamiento:', style=style),
    widgets.IntSlider(value=0, min=0, max=100, step=1, description='Ensayos de Preexposición (Inhibición Latente):', style=style),
    widgets.FloatLogSlider(value=1.0, base=10, min=-1, max=2, step=0.1, description='Incertidumbre Inicial (σ²₀):', style=style),
    widgets.FloatLogSlider(value=0.01, base=10, min=-4, max=-1, step=0.1, description='Volatilidad del Entorno (τ²):', style=style),
    widgets.FloatLogSlider(value=0.1, base=10, min=-2, max=1, step=0.1, description='Ruido de la Observación (σᵣ²):', style=style)
])

# --- Simulador 2: Teoría de la Información (Ley C/T) ---

def simulate_info_theory(T, ITI):
    """
    Calcula la informatividad y la predicción de ensayos para la adquisición.
    """
    if T <= 0:
        print("El intervalo T (CS-US) debe ser mayor que cero.")
        return

    C = T + ITI
    C_over_T = C / T

    # El parámetro k es el intercepto de la regresión de Harris & Gallistel (2024),
    # que representa la informatividad que produce aprendizaje en 1 ensayo. Un valor plausible es ~250-300.
    k_intercept = 280

    if C_over_T <= 1:
        n_acq_str = "Infinito (no hay contingencia positiva)."
    else:
        n_acq = k_intercept / (C_over_T - 1)
        n_acq_str = f"{n_acq:.1f}"

    print("--- Resultados de la Teoría de la Información ---")
    print(f"Intervalo CS-US (T): {T} s")
    print(f"Intervalo Inter-Ensayo (ITI): {ITI} s")
    print(f"Intervalo Ciclo-a-Ciclo (C = T + ITI): {C} s")
    print("-------------------------------------------------")
    print(f"Informatividad (C/T): {C_over_T:.2f}")
    print(f"Ensayos predichos para la adquisición (n_acq): {n_acq_str}")
    print("\n*Nota: Un valor de C/T alto significa que el CS es muy informativo y el aprendizaje es más rápido.")

# Creación de los Widgets para Teoría de la Información
info_widgets = VBox([
    widgets.IntSlider(value=10, min=1, max=100, step=1, description='Intervalo CS-US (T):', style=style),
    widgets.IntSlider(value=90, min=1, max=1000, step=1, description='Intervalo Inter-Ensayo (ITI):', style=style)
])


# --- Visualización de los Simuladores ---

# Pestañas para organizar los simuladores
tabs = widgets.Tab()

# Simulador de Kalman
kalman_out = interactive_output(simulate_kalman, {'n_trials': kalman_widgets.children[0],
                                                'n_preexposure': kalman_widgets.children[1],
                                                'initial_uncertainty': kalman_widgets.children[2],
                                                'volatility': kalman_widgets.children[3],
                                                'noise': kalman_widgets.children[4]})
tab_kalman = VBox([widgets.HTML("<h2>Simulador del Filtro de Kalman</h2>"), kalman_widgets, kalman_out])

# Simulador de Teoría de la Información
info_out = interactive_output(simulate_info_theory, {'T': info_widgets.children[0], 'ITI': info_widgets.children[1]})
tab_info = VBox([widgets.HTML("<h2>Simulador de la Teoría de la Información (Ley C/T)</h2>"), info_widgets, info_out])

tabs.children = [tab_kalman, tab_info]
tabs.set_title(0, 'Filtro de Kalman')
tabs.set_title(1, 'Teoría de la Información')

# Mostrar todo
display(tabs)

Guía para los Estudiantes (Python)
Simulador del Filtro de Kalman:

Este simulador muestra cómo un agente bayesiano aprende sobre un estímulo.

Gráfico Superior (Azul): Muestra la fuerza asociativa estimada (
hatw). Observa cómo crece hasta alcanzar el valor de la recompensa (1).

Gráfico del Medio (Verde): Muestra la incertidumbre (
sigma
2
 ). Fíjate cómo la incertidumbre disminuye a medida que el agente aprende.

Gráfico Inferior (Morado): Muestra la Ganancia de Kalman (k), o tasa de aprendizaje. Es alta al principio (cuando hay mucha incertidumbre) y disminuye a medida que el agente gana confianza.

Prueba esto:

Aumenta los "Ensayos de Preexposición" a 50. Verás el fenómeno de Inhibición Latente. El aprendizaje en la fase de condicionamiento será mucho más lento porque la incertidumbre inicial ya era muy baja, lo que aplana la curva de la Ganancia de Kalman.

Aumenta la "Volatilidad del Entorno" (
tau
2
 ). Esto le dice al agente que el mundo cambia, por lo que nunca deja de aprender. La Ganancia de Kalman no caerá a cero y el sistema seguirá siendo sensible a nuevos datos.

Simulador de la Teoría de la Información:

Este simulador no es una simulación ensayo a ensayo, sino una calculadora basada en la ley C/T de Gallistel, Harris y Gibbon.

Muestra cómo la informatividad del estímulo determina la velocidad de aprendizaje predicha.

Prueba esto:

Fija T=10 y ITI=90. Esto da C=100 y C/T=10. Anota los ensayos predichos.

Ahora, mantén la relación C/T=10 pero cambia los valores absolutos. Por ejemplo, T=50 y ITI=450 (C=500). Verás que el número de ensayos predichos es casi el mismo. Esto demuestra la invarianza de escala temporal.

Maximiza la informatividad: pon T en su valor más bajo (1) y el ITI en su valor más alto (1000). Verás que el aprendizaje predicho es extremadamente rápido, acercándose a un solo ensayo.

